In [1]:
import ipyleaflet
import polyline as pl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ipyleaflet import Map, Marker
from ipywidgets import Button
from IPython.display import display, HTML
import googlemaps
import numpy as np
from ipyleaflet import AwesomeIcon,Icon
import json

class VRPSolver:
    def __init__(self, center, vehicle_capacities, zoom=12):
        with open('credentials.json') as f:
            credentials = json.load(f)
        self.API_KEY = credentials["API_KEY"]
        self.center = center
        self.zoom = zoom
        self.vehicle_capacities = vehicle_capacities

        self.default_icon = AwesomeIcon(name="circle", marker_color='blue', icon_color='black')
        self.special_icon = AwesomeIcon(name="circle", marker_color='red', icon_color='black')
        self.title = HTML(
            '<h1>Click on the map to select locations. <br>'
            'The first location is your starting point (warehouse) <br> '
            'each subsequent point is a delivery destination.</h1>'
        )
        display(self.title)

        # Initialize the Google Maps client
        self.gmaps = googlemaps.Client(key=self.API_KEY)

        # Remove all layers and listeners from the old map
        if hasattr(self, 'm'):
            for layer in self.m.layers:
                self.m.remove_layer(layer)
            self.m.clear_layers()
            self.m.clear_callbacks()

        self.m = Map(center=self.center, zoom=self.zoom)
        self.markers = []
        self.polylines = []   # initialize the polylines attribute
        self.m.on_interaction(self.handle_click)

        # Remove the old button
        if hasattr(self, 'button'):
            self.delete_markers_and_lines.close()

        self.delete_markers_and_lines = Button(description="Delete all markers")
        self.delete_markers_and_lines.on_click(self.clear_markers)

        self.distance_matrix_button = Button(description="Calculate distance matrix")
        self.distance_matrix_button.on_click(self.calculate_distance_matrix)
        
        self.solve_vrp_button = Button(description="Solve VRP")
        self.solve_vrp_button.on_click(self.solve_vrp)

        display(self.delete_markers_and_lines)
        display(self.distance_matrix_button)
        display(self.solve_vrp_button)
        display(self.m)

    def handle_click(self, **kwargs):
        if kwargs.get('type') == 'click':
            # If there is a special marker, change it back to a default marker
            if len(self.markers) == 0:
                marker = Marker(location=kwargs.get('coordinates'), icon=self.special_icon)
            else:
                marker = Marker(location=kwargs.get('coordinates'), icon=self.default_icon)            

            # Add the new marker to the map and to our list
            self.m.add_layer(marker)
            self.markers.append(marker)

            # If we have more than 10 markers, remove the oldest one
            if len(self.markers) > 10:
                self.m.remove_layer(self.markers.pop(0))
                old_marker = self.markers[0]
                new_marker = Marker(location=old_marker.location, icon=self.special_icon)
                self.markers[0] =new_marker
                self.m.substitute_layer(old_marker, new_marker)

    def set_vehicle_capacities(self, vehicle_capacities):
        self.vehicle_capacities = vehicle_capacities
        
    def clear_markers(self, button):
        print("Clearing markers")
        for marker in self.markers:
            self.m.remove_layer(marker)
        self.markers.clear()
        self.markers = []

        for polyline in self.polylines:
            self.m.remove_layer(polyline)
        self.polylines.clear()
        self.polylines = []

    def calculate_distance_matrix(self, button):
        if self.markers == []:
            print("No markers set")
            return
        else:
            # Get the coordinates from the markers
            coordinates = [marker.location for marker in self.markers]

            # Get the distances between all pairs of points
            num_points = len(coordinates)
            self.distance_matrix = np.zeros((num_points, num_points))

            for i in range(num_points):
                for j in range(i+1, num_points):
                    origin = coordinates[i]
                    destination = coordinates[j]

                    # Request the distance between origin and destination from Google Maps API
                    result = self.gmaps.distance_matrix(origins=[origin],
                                                        destinations=[destination],
                                                        mode="driving")

                    # Extract the distance from the result (in meters)
                    distance = result['rows'][0]['elements'][0]['distance']['value']

                    # Save the distance in our matrix
                    self.distance_matrix[i, j] = round(distance)
                    self.distance_matrix[j, i] = round(distance)  # The distance matrix is symmetric

            self.distance_matrix = self.distance_matrix.astype(int)
            print(self.distance_matrix)  # Print the distance matrix

    def solve_vrp(self, button):
            # Define the demand for each location (number of parcels to be delivered)
            self.demands = [0] + np.random.randint(1,50,len(self.markers)-1).tolist()  # The first location is the depot (no demand)

            print(self.demands)

            # Create the routing index manager
            self.manager = pywrapcp.RoutingIndexManager(len(self.distance_matrix),
                                            len(self.vehicle_capacities), 0)

            # Create Routing Model
            self.routing = pywrapcp.RoutingModel(self.manager)

            def demand_callback(from_index):
                """Returns the demand of the node."""
                from_node = self.manager.IndexToNode(from_index)
                return self.demands[from_node]

            demand_callback_index = self.routing.RegisterUnaryTransitCallback(demand_callback)

            def distance_callback(from_index, to_index):
                from_node = self.manager.IndexToNode(from_index)
                to_node = self.manager.IndexToNode(to_index)
                return self.distance_matrix[from_node][to_node]

            transit_callback_index = self.routing.RegisterTransitCallback(distance_callback)

            # Define cost of each arc
            self.routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

            # Add Capacity constraint
            self.routing.AddDimensionWithVehicleCapacity(
                demand_callback_index,
                0,  # null capacity slack
                self.vehicle_capacities,  # vehicle maximum capacities
                True,  # start cumul to zero
                "Capacity"
            )

            # Set the parameters of the search
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = (
                routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
            )

            # Solve the problem
            self.solution = self.routing.SolveWithParameters(search_parameters)
            
            if self.solution:
                self.print_solution()
                self.visualize_routes()
            else:
                print('No solution found !')

    def print_solution(self):
        max_route_distance = 0
        for vehicle_id in range(self.routing.vehicles()):
            index = self.routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
            route_distance = 0
            while not self.routing.IsEnd(index):
                plan_output += ' {} ->'.format(self.manager.IndexToNode(index))
                previous_index = index
                index = self.solution.Value(self.routing.NextVar(index))
                route_distance += self.routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            plan_output += ' {}\n'.format(self.manager.IndexToNode(index))
            plan_output += 'Distance of the route: {}\n'.format(route_distance)
            print(plan_output)
            max_route_distance = max(route_distance, max_route_distance)
        print('Maximum of the route distances: {}'.format(max_route_distance))

    def visualize_routes(self):
        # Extract the routes from the solution
        self.routes = []
        colors = ["blue", "green", "red", "purple", "orange", "black"]

        for vehicle_id in range(len(self.vehicle_capacities)):
            route = []
            index = self.routing.Start(vehicle_id)
            while not self.routing.IsEnd(index):
                node_index = self.manager.IndexToNode(index)
                route.append(node_index)
                index = self.solution.Value(self.routing.NextVar(index))
            self.routes.append(route)

        self.polylines = []
        coordinates = [marker.location for marker in self.markers]

        for vehicle_id, route in enumerate(self.routes):
            for i in range(len(route)-1):
                origin = coordinates[route[i]]
                destination = coordinates[route[i+1]]

                # Request directions via the Google Maps Directions API
                result = self.gmaps.directions(origin=origin,
                                            destination=destination,
                                            mode='driving')

                # The result contains a route as an encoded polyline
                encoded_route = result[0]['overview_polyline']['points']

                # Decode the polyline to get a list of lat, lon pairs
                route_coordinates = pl.decode(encoded_route)

                # Create a polyline and add it to the map
                polyline_layer = ipyleaflet.Polyline(
                    locations=route_coordinates,
                    color=colors[vehicle_id % len(colors)],
                    fill=False
                )
                self.m.add_layer(polyline_layer)
                self.polylines.append(polyline_layer)

solver = VRPSolver(center=(53.561520361897486, 10.003738403320314), vehicle_capacities=[50,50,50,50,50,50,50])


Button(description='Delete all markers', style=ButtonStyle())

Button(description='Calculate distance matrix', style=ButtonStyle())

Button(description='Solve VRP', style=ButtonStyle())

Map(center=[53.561520361897486, 10.003738403320314], controls=(ZoomControl(options=['position', 'zoom_in_text'…

[[   0 2955 2581]
 [2955    0 3402]
 [2581 3402    0]]
[0, 10, 26]
Route for vehicle 0:
 0 -> 0
Distance of the route: 0

Route for vehicle 1:
 0 -> 0
Distance of the route: 0

Route for vehicle 2:
 0 -> 0
Distance of the route: 0

Route for vehicle 3:
 0 -> 0
Distance of the route: 0

Route for vehicle 4:
 0 -> 0
Distance of the route: 0

Route for vehicle 5:
 0 -> 0
Distance of the route: 0

Route for vehicle 6:
 0 -> 2 -> 1 -> 0
Distance of the route: 8938

Maximum of the route distances: 8938
Clearing markers
